In [43]:
%reload_ext autoreload
%autoreload 2

In [26]:
import os
os.getcwd()

'/Users/sumitgulati/Documents/POC_Text2Sql_V1'

In [4]:
# import sys
# from pathlib import Path

# # Set project root (adjust if notebook location differs)
# PROJECT_ROOT = Path("/Users/chandrashekharburadkar/Desktop/POC_AMADEUS/garv-amadeus-text2sql")
# sys.path.append(str(PROJECT_ROOT))

# print("Project root added:", PROJECT_ROOT)

Project root added: /Users/chandrashekharburadkar/Desktop/POC_AMADEUS/garv-amadeus-text2sql


In [2]:
import sys
!{sys.executable} -m pip install duckdb python-dotenv --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import duckdb
duckdb.__version__

'1.1.3'

In [4]:
import sys
!{sys.executable} -m pip install numpy pandas --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd

In [27]:
from app.db.duckdb_client import get_conn
conn = get_conn()

conn.execute("""
SELECT airport, AVG(avg_wait_min) AS avg_wait
FROM presecurity_events
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
""").df()

,airport,avg_wait
0,DXB,7.546644
1,LHR,6.716024
2,CDG,6.045573
3,FRA,5.513570
4,AMS,5.385072


In [15]:
import sys
!{sys.executable} -m pip install duckdb python-dotenv pyarrow

  Using cached pyarrow-22.0.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (3.2 kB)
Using cached pyarrow-22.0.0-cp313-cp313-macosx_12_0_arm64.whl (34.2 MB)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


In [17]:
from app.db.duckdb_client import get_conn
conn = get_conn()

conn.execute("PRAGMA database_list").df()

,seq,name,file
0,629,amadeus_ops,/Users/chandrashekharburadkar/Desktop/POC_AMAD...


In [18]:
conn.execute("SHOW TABLES").df()


,name
0,boarding_events
1,checkin_events
2,dim_airport
3,disruption_events
4,flights
5,presecurity_events


In [4]:
conn.close()

In [1]:
from app.db.duckdb_client import get_conn
conn = get_conn()
conn.execute("SHOW TABLES").df()

,name
0,boarding_events
1,checkin_events
2,dim_airport
3,disruption_events
4,flights
5,gold_airport_kpi_hourly
6,gold_anomaly_scores
7,gold_delay_reason_daily
8,presecurity_events


In [2]:
from app.db.duckdb_client import get_conn

conn = get_conn()

conn.execute("""
SELECT airport,
       hour,
       security_wait_min,
       avg_queue_len
FROM gold_airport_kpi_hourly
ORDER BY security_wait_min DESC
LIMIT 10
""").df()

,airport,hour,security_wait_min,avg_queue_len
0,LHR,2025-12-23 12:00:00,12.809167,95.666667
1,DXB,2025-12-23 18:00:00,12.215833,136.166667
2,CDG,2025-12-29 10:00:00,11.560000,137.000000
3,DXB,2025-12-24 12:00:00,11.320000,90.083333
4,DXB,2025-12-27 11:00:00,10.881667,80.250000
5,DXB,2025-12-27 05:00:00,10.679167,74.750000
6,DXB,2025-12-26 21:00:00,10.584167,67.666667
7,DXB,2025-12-25 15:00:00,10.466667,84.083333
8,DXB,2025-12-24 14:00:00,10.375833,84.666667
9,AMS,2025-12-29 05:00:00,10.212500,59.416667


In [3]:
conn.execute("""
SELECT airport, AVG(avg_wait_min) AS avg_wait
FROM presecurity_events
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
""").df()

,airport,avg_wait
0,DXB,7.546644
1,LHR,6.716024
2,CDG,6.045573
3,FRA,5.513570
4,AMS,5.385072


In [5]:
# from app.rag.schema_retriever import retrieve_relevant_schema

# out = retrieve_relevant_schema("Which airport has highest average security wait time?")
# out["relevant_tables"], out["schema_context"][:500]

In [28]:
import os

# Make sure env vars are set
os.environ["DUCKDB_PATH"] = "data/amadeus_ops.duckdb"
os.environ["DUCKDB_READ_ONLY"] = "true"

print("Env ready")

Env ready


In [7]:
from app.db.duckdb_client import get_conn

conn = get_conn()

conn.execute("SHOW TABLES").df()

,name
0,boarding_events
1,checkin_events
2,dim_airport
3,disruption_events
4,flights
5,gold_airport_kpi_hourly
6,gold_anomaly_scores
7,gold_delay_reason_daily
8,presecurity_events


In [8]:
conn.execute("""
SELECT *
FROM gold_airport_kpi_hourly
LIMIT 5
""").df()

,airport,hour,checkin_wait_min,pax_volume,security_wait_min,avg_lanes_open,avg_queue_len,boarding_delay_min
0,LHR,2025-12-22 20:00:00,7.279167,1948.0,5.480833,10.416667,79.750000,8.585
1,LHR,2025-12-23 21:00:00,8.339167,1282.0,8.233333,6.000000,67.583333,2.770
2,LHR,2025-12-23 23:00:00,8.677500,1341.0,9.103333,6.166667,76.333333,4.970
3,LHR,2025-12-25 00:00:00,8.763333,856.0,7.206667,4.250000,42.916667,4.780
4,LHR,2025-12-25 03:00:00,8.581667,840.0,6.983333,4.250000,42.083333,0.000


In [1]:
from langchain.schema import Document

In [2]:
from app.rag.schema_docs import extract_schema_docs

docs = extract_schema_docs()

len(docs)

9

In [3]:
from app.db.duckdb_client import get_conn

conn = get_conn()

conn.execute("SHOW TABLES").df()

,name
0,boarding_events
1,checkin_events
2,dim_airport
3,disruption_events
4,flights
5,gold_airport_kpi_hourly
6,gold_anomaly_scores
7,gold_delay_reason_daily
8,presecurity_events


In [4]:
conn.execute("""
SELECT *
FROM gold_airport_kpi_hourly
ORDER BY security_wait_min DESC
LIMIT 5
""").df()

,airport,hour,checkin_wait_min,pax_volume,security_wait_min,avg_lanes_open,avg_queue_len,boarding_delay_min
0,LHR,2025-12-23 12:00:00,8.250000,1397.0,12.809167,5.833333,95.666667,7.660
1,DXB,2025-12-23 18:00:00,9.322500,2169.0,12.215833,9.000000,136.166667,6.020
2,CDG,2025-12-29 10:00:00,6.200000,157.0,11.560000,10.000000,137.000000,NaN
3,DXB,2025-12-24 12:00:00,7.713333,1477.0,11.320000,7.250000,90.083333,1.465
4,DXB,2025-12-27 11:00:00,7.455833,1372.0,10.881667,5.500000,80.250000,NaN


In [6]:
conn.execute("""
SELECT
    airport,
    day,
    top_reason,
    top_reason_count,
    top_reason_avg_delay_min
FROM gold_delay_reason_daily
ORDER BY top_reason_avg_delay_min DESC
LIMIT 10
""").df()

,airport,day,top_reason,top_reason_count,top_reason_avg_delay_min
0,FRA,2025-12-23,SEC,7,24.862857
1,DXB,2025-12-26,GATE,9,18.664444
2,LHR,2025-12-24,GATE,11,17.667273
3,CDG,2025-12-29,BAG,6,16.870000
4,AMS,2025-12-27,TECH,7,15.424286
5,LHR,2025-12-28,CREW,9,15.233333
6,AMS,2025-12-28,TECH,9,14.633333
7,CDG,2025-12-26,ATC,10,12.952000
8,LHR,2025-12-22,ATC,5,11.636000
9,FRA,2025-12-29,BAG,7,11.375714


In [6]:
# conn.execute("""
# SELECT
#   airport,
#   hour,
#   metric_name,
#   metric_value,
#   z_score
# FROM gold_anomaly_scores
# ORDER BY z_score DESC
# LIMIT 10
# """).df()

In [29]:
from app.rag.schema_docs import extract_schema_docs

docs = extract_schema_docs()
len(docs), docs[0].page_content[:300], docs[0].metadata

(9,
 'Table: boarding_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- flight_id (VARCHAR)\n- boarding_delay_min (DOUBLE)\n- reason_code (VARCHAR)',
 {'table': 'boarding_events'})

In [10]:
import app.rag.schema_index as si
[x for x in dir(si) if "index" in x or "load" in x or "schema" in x]

['__loader__',
 'build_schema_index',
 'extract_schema_docs',
 'get_schema_vectorstore']

In [30]:
from app.rag.schema_index import build_schema_index
vs = build_schema_index()

results = vs.similarity_search("Top 5 airports by average security wait time", k=5)
[(r.metadata.get("table"), r.page_content.splitlines()[0]) for r in results]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


[('presecurity_events', 'Table: presecurity_events'),
 ('presecurity_events', 'Table: presecurity_events'),
 ('presecurity_events', 'Table: presecurity_events'),
 ('presecurity_events', 'Table: presecurity_events'),
 ('presecurity_events', 'Table: presecurity_events')]

Checking query rewriter agnet

In [44]:
import sys
import os
from dotenv import load_dotenv

# Add project root to path
PROJECT_ROOT = os.path.abspath("..") if os.getcwd().endswith("notebooks") else os.getcwd()
sys.path.insert(0, PROJECT_ROOT)

# Load environment variables
load_dotenv()

print("PYTHONPATH OK")

PYTHONPATH OK


In [45]:
import langchain
import langchain_core
import langsmith

print("langchain:", langchain.__version__)
print("langchain_core:", langchain_core.__version__)
print("langsmith:", langsmith.__version__)

langchain: 0.3.14
langchain_core: 1.2.5
langsmith: 0.5.2


In [49]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [50]:
print("LLM_PROVIDER =", os.getenv("LLM_PROVIDER"))
print("OLLAMA_MODEL =", os.getenv("OLLAMA_MODEL"))


LLM_PROVIDER = gemini
OLLAMA_MODEL = None


In [51]:
from dotenv import load_dotenv
load_dotenv()

from app.agents.query_rewriter import rewrite_query

rewrite_query("Top 5 airports by average security wait time last 7 days")

{'rewritten_query': 'Calculate the average security wait time for each airport over the period spanning the last 7 complete days, and return the top 5 airports ranked in descending order of this average wait time.',
 'intent': 'RANKING',
 'entities': {'metric': 'Security Wait Time',
  'aggregation': 'Average',
  'time_period': 'Last 7 days',
  'group_by': 'Airport',
  'limit': 5},
 'clarification_needed': False,
 'clarification_question': '',
 'notes': 'A straightforward ranking request based on a specific metric (average security wait time) and a defined rolling time window.',
 'timestamp_utc': '2025-12-31T13:04:46.328128'}

In [7]:
# vs = build_schema_index()
# docs = vs.similarity_search(rew["rewritten_query"], k=5)
# schema_context = "\n\n".join([d.page_content for d in docs])

In [ ]:

out = generate_sql(
    rewritten_query=rew["rewritten_query"],
    intent=rew["intent"],
    entities=rew["entities"],
    schema_context=schema_context,
)

out["sql"], out["warnings"], out["confidence"]

In [11]:
import os, sys
from dotenv import load_dotenv

# Load env variables
load_dotenv(override=True)

# Ensure project root is on path
sys.path.insert(0, os.getcwd())

# Imports (THIS WAS MISSING)
from app.rag.schema_index import build_schema_index
from app.agents.query_rewriter import rewrite_query
from app.agents.sql_generator import generate_sql

In [12]:
rew = rewrite_query("Top 5 airports by average security wait time last 7 days")

vs = build_schema_index()
docs = vs.similarity_search(rew["rewritten_query"], k=5)

schema_context = "\n\n".join([d.page_content for d in docs])
schema_context[:500]

Query rewrite failed
Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/langchain_core/output_parsers/pydantic.py", line 28, in _parse_obj
    return self.pydantic_object.model_validate(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/pydantic/main.py", line 627, in model_validate
    return cls.__pydantic_validator__.validate_python(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pydantic_core._pydantic_core.ValidationError: 1 validation error for QueryRewriteOutput
clarification_question
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql_V1/app/

'Table: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: p'

In [13]:
# 1️⃣ Rewrite
rew = rewrite_query("Top 5 airports ")
rew

{'rewritten_query': 'SELECT airport_name, COUNT(*) AS flight_count FROM flights GROUP BY airport_name ORDER BY flight_count DESC LIMIT 5',
 'intent': 'RANKING',
 'entities': {'metric': 'number of flights', 'limit': 5},
 'clarification_needed': True,
 'clarification_question': 'What metric would you like to rank the airports by? (e.g., number of flights, passenger volume, on-time performance)',
 'notes': "The user asked for 'Top 5 airports' without specifying a metric. Assuming 'number of flights' as a default, but clarification is needed to confirm the desired metric for ranking.",
 'timestamp_utc': '2025-12-30T16:26:49.287204'}

In [14]:
# 1️⃣ Rewrite
rew = rewrite_query("Top 5 airports by average security wait time last 7 days")

Query rewrite failed
Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/langchain_core/output_parsers/pydantic.py", line 28, in _parse_obj
    return self.pydantic_object.model_validate(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/pydantic/main.py", line 627, in model_validate
    return cls.__pydantic_validator__.validate_python(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pydantic_core._pydantic_core.ValidationError: 1 validation error for QueryRewriteOutput
clarification_question
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql_V1/app/

In [15]:
rew


{'rewritten_query': 'Top 5 airports by average security wait time last 7 days',
 'intent': 'UNKNOWN',
 'entities': {},
 'clarification_needed': False,
 'clarification_question': '',
 'notes': 'Rewrite failed: Failed to parse QueryRewriteOutput from completion {"rewritten_query": "SELECT airport_name, AVG(security_wait_time) AS average_wait_time FROM security_wait_times WHERE date >= DATE(\'now\', \'-7 days\') GROUP BY airport_name ORDER BY average_wait_time DESC LIMIT 5", "intent": "RANKING", "entities": {"metric": "average security wait time", "time_range": "last 7 days", "limit": 5}, "clarification_needed": false, "clarification_question": null, "notes": "The user is asking for a ranking of airports based on a specific metric (average security wait time) over a defined period (last 7 days) and wants the top N results (top 5)."}. Got: 1 validation error for QueryRewriteOutput\nclarification_question\n  Input should be a valid string [type=string_type, input_value=None, input_type=None

In [16]:
# 2️⃣ Schema RAG
vs = build_schema_index()
docs = vs.similarity_search(rew["rewritten_query"], k=5)
print(docs[0].page_content)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)


In [17]:
schema_context = "\n\n".join([d.page_content for d in docs])
print(schema_context)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)


In [12]:

# 3️⃣ SQL generation
out = generate_sql(
    rewritten_query=rew["rewritten_query"],
    intent=rew["intent"],
    entities=rew["entities"],
    schema_context=schema_context,
)

print(out["sql"])
print(out["warnings"])

SELECT airport, AVG(security_wait_min) AS avg_security_wait_time FROM gold_airport_kpi_hourly WHERE hour >= NOW() - INTERVAL '7 days' GROUP BY airport ORDER BY avg_security_wait_time DESC LIMIT 5;
[]


In [13]:
display(out["sql"])

"SELECT airport, AVG(security_wait_min) AS avg_security_wait_time FROM gold_airport_kpi_hourly WHERE hour >= NOW() - INTERVAL '7 days' GROUP BY airport ORDER BY avg_security_wait_time DESC LIMIT 5;"

In [41]:
import os, sys
from dotenv import load_dotenv
load_dotenv(override=True)
sys.path.insert(0, os.getcwd())

from app.rag.schema_index import build_schema_index
from app.agents.query_rewriter import rewrite_query
from app.agents.sql_generator import generate_sql
from app.agents.sql_validator import validate_and_autofix_sql
from app.db.duckdb_client import get_conn

# 1) Rewrite
rew = rewrite_query("Top 5 airports by average security wait time last 7 days")

# 2) Schema RAG
vs = build_schema_index()
docs = vs.similarity_search(rew["rewritten_query"], k=5)
schema_context = "\n\n".join([d.page_content for d in docs])
display(schema_context)

# 3) Generate SQL (Step 6)
gen = generate_sql(
    rewritten_query=rew["rewritten_query"],
    intent=rew["intent"],
    entities=rew["entities"],
    schema_context=schema_context,
)
candidate_sql = gen["sql"]
print("CANDIDATE SQL:\n", candidate_sql)

# 4) Validate + Auto-fix (Step 7)
val = validate_and_autofix_sql(
    rewritten_query=rew["rewritten_query"],
    schema_context=schema_context,
    candidate_sql=candidate_sql,
    max_retries=1,
)

print("\nVALIDATION RESULT:\n", val)

# 5) Execute only if valid
if val["ok"]:
    conn = get_conn()
    df = conn.execute(val["final_sql"]).df()
    df.head()


Query rewrite failed
Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/sumitgulati/Documents/POC_Te

'Table: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)\n\nTable: presecurity_events\nColumns:\n- airport (VARCHAR)\n- ts (TIMESTAMP)\n- pax_count (INTEGER)\n- avg_wait_min (DOUBLE)\n- lanes_open (INTEGER)\n- queue_len (INTEGER)'

SQL generation failed
Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/sumitgulati/Documents/POC_T

CANDIDATE SQL:
 


Exception ignored in: <function Channel.__del__ at 0x13dd1ac00>
Traceback (most recent call last):
  File "/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/grpc/_channel.py", line 2225, in __del__
KeyboardInterrupt: 


RuntimeError: Ollama HTTP call failed. Check Ollama is running at http://localhost:11434 and model 'qwen2.5:7b' exists. Original error: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x178d62490>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [3]:
from app.agents.explainer import explain_answer
from app.db.duckdb_client import get_conn

conn = get_conn()

sql = """
SELECT airport,
       AVG(security_wait_min) AS avg_security_wait_time
FROM gold_airport_kpi_hourly
WHERE hour >= NOW() - INTERVAL '7 days'
GROUP BY airport
ORDER BY avg_security_wait_time DESC
LIMIT 5
"""

df = conn.execute(sql).df()

display(df)

,airport,avg_security_wait_time
0,DXB,7.531699
1,LHR,6.698814
2,CDG,6.032511
3,FRA,5.500529
4,AMS,5.364872


In [6]:
result = explain_answer(
    user_question="Top 5 airports by average security wait time last 7 days",
    sql=sql,
    df=df
)

print(result["answer"])

Based on the data from the last 7 days, here are the top 5 airports by average security wait time:

1. **DXB (Dubai International Airport)** - Average security wait time of 7.53 minutes.
2. **LHR (London Heathrow Airport)** - Average security wait time of 6.70 minutes.
3. **CDG (Paris Charles de Gaulle Airport)** - Average security wait time of 6.03 minutes.
4. **FRA (Frankfurt Airport)** - Average security wait time of 5.50 minutes.
5. **AMS (Amsterdam Schiphol Airport)** - Average security wait time of 5.36 minutes.

The results show that the average security wait times at these airports are relatively short, with AMS having the shortest average wait time among this group. DXB has the longest average wait time, indicating it may have more congestion or longer lines during peak hours compared to other airports in this list.


In [7]:
from app.agents.schema_agent import get_schema_context
sc = get_schema_context("Top 5 airports by average security wait time last 7 days", k=5)
print(sc["schema_context"][:400])

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)
- pax_count (INTEGER)
- avg_wait_min (DOUBLE)
- lanes_open (INTEGER)
- queue_len (INTEGER)

Table: presecurity_events
Columns:
- airport (VARCHAR)
- ts (TIMESTAMP)



In [8]:
from app.agents.visualizer import suggest_plot
viz = suggest_plot(df)   # df from sql_executor
viz

{'plot': 'bar',
 'x': 'airport',
 'y': 'avg_security_wait_time',
 'reason': 'Category + numeric detected'}

In [9]:
! python scripts/print_project_structure.py


PROJECT STRUCTURE:

/Users/chandrashekharburadkar/Desktop/POC_AMADEUS/garv-amadeus-text2sql
├── .env
├── .env.example
├── .gitignore
├── README.md
├── app
│   ├── __init__.py
│   ├── agents
│   │   ├── __init__.py
│   │   ├── explainer.py
│   │   ├── llm_factory.py
│   │   ├── query_rewriter.py
│   │   ├── schema_agent.py
│   │   ├── sql_executor.py
│   │   ├── sql_generator.py
│   │   ├── sql_validator.py
│   │   └── visualizer.py
│   ├── api
│   │   └── routes.py
│   ├── audit
│   │   └── langsmith_tracing.py
│   ├── auth
│   ├── config
│   │   ├── llm.py
│   │   └── settings.py
│   ├── db
│   │   └── duckdb_client.py
│   ├── fakedb
│   │   └── build_fake_db.py
│   ├── graph
│   │   └── text2sql_graph.py
│   ├── main.py
│   ├── pipelines
│   │   ├── 01_create_silver_tables.py
│   │   ├── 02_build_gold_tables.py
│   │   └── 03_build_schema_index.py
│   ├── rag
│   │   ├── __init__.py
│   │   ├── embeddings_factory.py
│   │   ├── schema_docs.py
│   │   ├── schema_index.py
│   │   └── 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# from dotenv import load_dotenv
# load_dotenv(override=True)

# from app.graph.text2sql_graph import run_text2sql
# out = run_text2sql("Top 5 airports by average security wait time last 7 days")
# out["candidate_sql"], out["final_sql"], out["validation_ok"]

In [9]:
# from dotenv import load_dotenv
# load_dotenv(override=True)

# from app.graph.text2sql_graph import run_text2sql
# out = run_text2sql("Top 5 airports by average security wait time last 7 days")

# print(out["final_sql"])



In [3]:
out

{'ok': True,
 'intent': 'RANKING',
 'entities': {'metric': 'average security wait time',
  'time_period': 'last 7 days'},
 'rewritten_query': 'What are the top 5 airports by average security wait time over the last 7 days?',
 'retrieved_tables': ['gold_airport_kpi_hourly'],
 'candidate_sql': {'sql': "SELECT airport, AVG(security_wait_min) AS avg_security_wait_min FROM gold_airport_kpi_hourly WHERE hour >= NOW() - INTERVAL '7 days' GROUP BY airport ORDER BY avg_security_wait_min DESC LIMIT 5;",
  'used_tables': ['gold_airport_kpi_hourly'],
  'used_columns': ['airport', 'security_wait_min', 'hour'],
  'assumptions': ['Time window assumed to be the last 7 days based on the provided schema context.'],
  'warnings': [],
  'confidence': 1.0,
  'timestamp_utc': '2025-12-29T18:17:11.843007'},
 'final_sql': "SELECT airport, AVG(security_wait_min) AS avg_security_wait_min FROM gold_airport_kpi_hourly WHERE hour >= NOW() - INTERVAL '7 days' GROUP BY airport ORDER BY avg_security_wait_min DESC LIM

In [4]:

out["dataframe"]["df"].head()


,airport,avg_security_wait_min
0,DXB,7.526344
1,LHR,6.696048
2,CDG,6.024237
3,FRA,5.502075
4,AMS,5.371027


In [7]:
print(out["explanation"]["summary"])

### Summary
The top 5 airports by average security wait time over the last 7 days are Dubai International Airport (DXB), Heathrow Airport (LHR), Charles de Gaulle Airport (CDG), Frankfurt Airport (FRA), and Amsterdam Schiphol Airport (AMS). These results indicate that passengers at these airports experience relatively shorter security lines on average.

### Key Insights
- **Dubai International Airport (DXB)** has the longest average security wait time among the top 5, with an average of approximately 7.53 minutes.
- **Heathrow Airport (LHR)** follows closely behind DXB, showing a slightly lower average wait time of about 6.70 minutes.
- **Charles de Gaulle Airport (CDG)** and **Frankfurt Airport (FRA)** have similar average wait times, both under 6 minutes, indicating efficient security operations at these airports.
- **Amsterdam Schiphol Airport (AMS)** has the shortest average wait time among this list, with an average of around 5.37 minutes.

### Assumptions and Caveats
- The data r

In [10]:
# import os, sys
# from dotenv import load_dotenv
# load_dotenv(override=True)

# # IMPORTANT: run notebook from project root OR add project root
# # sys.path.insert(0, os.getcwd())

# from app.graph.text2sql_graph import run_text2sql

# out = run_text2sql("Top 5 airports by average security wait time last 7 days")
# print(out["final_sql"])
# display(out["result_df"].head())
# print(out["preview_markdown"])
# print(out["explanation"]["summary"] if isinstance(out["explanation"], dict) else out["explanation"])

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

from app.graph.text2sql_graph import run_text2sql

out = run_text2sql("Top 5 airports by average security wait time last 7 days")
print(out["final_sql"])
display(out["result_df"].head())
print(out["preview_markdown"])

/Users/chandrashekharburadkar/Desktop/POC_AMADEUS/garv-amadeus-text2sql/app/rag/embeddings_factory.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/chandrashekharburadkar/Desktop/POC_AMADEUS/garv-amadeus-text2sql/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/chandrashekharburadkar/Desktop/POC_AMADEUS/garv-amadeus-text2sql/app/rag/schema_index.py:36: LangChainDeprec

SELECT airport, AVG(security_wait_min) AS avg_security_wait_time FROM gold_airport_kpi_hourly WHERE hour >= NOW() - INTERVAL '7 days' GROUP BY airport ORDER BY avg_security_wait_time DESC LIMIT 5;


,airport,avg_security_wait_time
0,DXB,7.526344
1,LHR,6.696048
2,CDG,6.024237
3,FRA,5.502075
4,AMS,5.371027


| airport   |   avg_security_wait_time |
|:----------|-------------------------:|
| DXB       |                  7.52634 |
| LHR       |                  6.69605 |
| CDG       |                  6.02424 |
| FRA       |                  5.50208 |
| AMS       |                  5.37103 |


In [53]:
from dotenv import load_dotenv
load_dotenv(override=True)


from app.graph.text2sql_graph import run_text2sql
out = run_text2sql("Top 5 airports by average security wait time last 7 days")
out.keys()

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 10e3554c-e2b5-483b-8b98-e5655ed6c079)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


dict_keys(['ok', 'intent', 'entities', 'rewritten_query', 'retrieved_tables', 'candidate_sql', 'final_sql', 'fixed_by_llm', 'dataframe', 'result_df', 'preview_markdown', 'explanation', 'chart_path', 'debug'])

In [54]:
!python -m app.main "Top 5 airports by average security wait time last 7 days"

python(69237) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/sumitgulati/Documents/POC_Text2Sql/.venv/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:47: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  from google.generativeai.caching import CachedContent  # type: ignore[import]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/sumitgulati/Documents/POC_Text2Sql_V1/app/main.py", line 79, in <module>
    main()
  File "/Users/sumitgulati/Documents/POC_Text2Sql_V1/app/main.py", line 75, in main
    run_cli(args.question)
  File "/Users/sumitgulati/Documents/POC_Text2Sql_V1/app/main.py", line 12, in run_cli
    out = run_text2sql(question)
          ^^^^^^^^

In [23]:
from app.graph.text2sql_graph import run_text2sql

out = run_text2sql("Top 5 airports by average security wait time last 7 days")
out["final_sql"]
out["result_df"]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


,airport,average_security_wait_time_minutes
0,DXB,7.529766
1,LHR,6.642244
2,CDG,6.022269
3,FRA,5.487288
4,AMS,5.399994
